In [1]:
import pandas as pd
import numpy as np
url = 'https://github.com/mattharrison/datasets/raw/master/data/vehicles.csv.zip'
df=pd.read_csv(url)
city_mpg = df.city08
highway_mpg = df.highway08
make = df.make
url = 'https://github.com/mattharrison/datasets/raw/master/data/alta-noaa-1980-2019.csv'
alta_df = pd.read_csv(url)
dates = pd.to_datetime(alta_df.DATE)
snow = (alta_df.SNOW.rename(dates))

/tmp/ipykernel_37177/2096392905.py:4: DtypeWarning: Columns (68,70,71,72,73,74,76,79) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(url)


# Categorical Manipulation
- labels that describe data
- for repeated values, if they have an intrinsic order, they are referred to as *oridinal* values
    - example: shirt sizes : small, medium, large
- underordered values such as colors are called *nominal* values
- can convert numerical data to categories by binning them

### 15.2 frequency counts
- use `.value_counts` method to determine *cardinality* of the values
- frequency of the values will tell you if a column is categorical
- if every value was unique or free form text, it is not categorical:


In [2]:
make.value_counts()

make
Chevrolet                      4003
Ford                           3371
Dodge                          2583
GMC                            2494
Toyota                         2071
                               ... 
Volga Associated Automobile       1
Panos                             1
Mahindra                          1
Excalibur Autos                   1
London Coach Co Inc               1
Name: count, Length: 136, dtype: int64

In [3]:
make.shape, make.nunique()

((41144,), 136)

### 15.3 Benfits of Categories
- uses less memory
- faster for many operations

In [4]:
cat_make = make.astype('category')

In [6]:
make.memory_usage(deep=True)

2606395

In [7]:
cat_make.memory_usage(deep = True)

95888

### 15.4 Conversion to Ordinal Categories
make an ordinal categorical (say in alphabetic order):

In [10]:
make_type = pd.CategoricalDtype(
    categories=sorted(make.unique()) , ordered=True)
ordered_make = make.astype(make_type)
ordered_make


0        Alfa Romeo
1           Ferrari
2             Dodge
3             Dodge
4            Subaru
            ...    
41139        Subaru
41140        Subaru
41141        Subaru
41142        Subaru
41143        Subaru
Name: make, Length: 41144, dtype: category
Categories (136, object): ['AM General' < 'ASC Incorporated' < 'Acura' < 'Alfa Romeo' ... 'Volvo' < 'Wallace Environmental' < 'Yugo' < 'smart']

- benefit of ordinal categoricals is that you can specify a lexical order to the items
- if the items have an order, can reduce operations like max and min, where you can specify an order rather than using an alphabetic order:

In [11]:
ordered_make.max()

'smart'

In [12]:
cat_make.max()

TypeError: Categorical is not ordered for operation max
you can use .as_ordered() to change the Categorical to an ordered one


In [13]:
ordered_make.sort_values()

20288    AM General
20289    AM General
369      AM General
358      AM General
19314    AM General
            ...    
31289         smart
31290         smart
29605         smart
22974         smart
26882         smart
Name: make, Length: 41144, dtype: category
Categories (136, object): ['AM General' < 'ASC Incorporated' < 'Acura' < 'Alfa Romeo' ... 'Volvo' < 'Wallace Environmental' < 'Yugo' < 'smart']

### 15.5 The `.cat` Accessor
- `.rename_categories`:

In [15]:
cat_make.cat.rename_categories(
    [c.lower() for c in cat_make.cat.categories])


0        alfa romeo
1           ferrari
2             dodge
3             dodge
4            subaru
            ...    
41139        subaru
41140        subaru
41141        subaru
41142        subaru
41143        subaru
Name: make, Length: 41144, dtype: category
Categories (136, object): ['am general', 'asc incorporated', 'acura', 'alfa romeo', ..., 'volvo', 'wallace environmental', 'yugo', 'smart']

In [16]:
ordered_make.cat.rename_categories(
    {c:c.lower() for c in ordered_make.cat.categories})


0        alfa romeo
1           ferrari
2             dodge
3             dodge
4            subaru
            ...    
41139        subaru
41140        subaru
41141        subaru
41142        subaru
41143        subaru
Name: make, Length: 41144, dtype: category
Categories (136, object): ['am general' < 'asc incorporated' < 'acura' < 'alfa romeo' ... 'volvo' < 'wallace environmental' < 'yugo' < 'smart']

The `.cat` attribute also allows you to add or remove categories and change the order of nomial categories.

In [17]:
ordered_make.cat.reorder_categories(
    sorted(cat_make.cat.categories, key=str.lower)
)

0        Alfa Romeo
1           Ferrari
2             Dodge
3             Dodge
4            Subaru
            ...    
41139        Subaru
41140        Subaru
41141        Subaru
41142        Subaru
41143        Subaru
Name: make, Length: 41144, dtype: category
Categories (136, object): ['Acura' < 'Alfa Romeo' < 'AM General' < 'American Motors Corporation' ... 'Volvo' < 'VPG' < 'Wallace Environmental' < 'Yugo']

### 15.6 Category Gotchas
- applying the `.value_counts` method or `.groupby` to categorical data uses all categories even if there were no values for them


In [18]:
ordered_make.iloc[:100].value_counts() #over 100 results because it includes every category

make
Dodge                        17
Oldsmobile                    8
Ford                          8
Buick                         7
Chevrolet                     5
                             ..
Grumman Allied Industries     0
Goldacre                      0
Geo                           0
Genesis                       0
smart                         0
Name: count, Length: 136, dtype: int64

In [19]:
(cat_make
 .iloc[:100]
 .groupby(cat_make.iloc[:100])
 .first()
 )

make
AM General                            NaN
ASC Incorporated                      NaN
Acura                                 NaN
Alfa Romeo                     Alfa Romeo
American Motors Corporation           NaN
                                  ...    
Volkswagen                     Volkswagen
Volvo                               Volvo
Wallace Environmental                 NaN
Yugo                                  NaN
smart                                 NaN
Name: make, Length: 136, dtype: category
Categories (136, object): ['AM General', 'ASC Incorporated', 'Acura', 'Alfa Romeo', ..., 'Volvo', 'Wallace Environmental', 'Yugo', 'smart']

This is a big issue when you group by two categories (or more) with dataframes and get a combinatoric explosion.

Compare this with the result from the string series:


In [20]:
(make
 .iloc[:100]
 .groupby(make.iloc[:100])
 .first()
 )

make
Alfa Romeo          Alfa Romeo
Audi                      Audi
BMW                        BMW
Buick                    Buick
CX Automotive    CX Automotive
Cadillac              Cadillac
Chevrolet            Chevrolet
Chrysler              Chrysler
Dodge                    Dodge
Ferrari                Ferrari
Ford                      Ford
Hyundai                Hyundai
Infiniti              Infiniti
Lexus                    Lexus
Mazda                    Mazda
Mercury                Mercury
Nissan                  Nissan
Oldsmobile          Oldsmobile
Plymouth              Plymouth
Pontiac                Pontiac
Rolls-Royce        Rolls-Royce
Subaru                  Subaru
Toyota                  Toyota
Volkswagen          Volkswagen
Volvo                    Volvo
Name: make, dtype: object

In [21]:
(make
 .iloc[:100]
 .groupby(make.iloc[:100], observed=True) #optional parameter which tells it to only include results for which there are values
 .first()
 )

make
Alfa Romeo          Alfa Romeo
Audi                      Audi
BMW                        BMW
Buick                    Buick
CX Automotive    CX Automotive
Cadillac              Cadillac
Chevrolet            Chevrolet
Chrysler              Chrysler
Dodge                    Dodge
Ferrari                Ferrari
Ford                      Ford
Hyundai                Hyundai
Infiniti              Infiniti
Lexus                    Lexus
Mazda                    Mazda
Mercury                Mercury
Nissan                  Nissan
Oldsmobile          Oldsmobile
Plymouth              Plymouth
Pontiac                Pontiac
Rolls-Royce        Rolls-Royce
Subaru                  Subaru
Toyota                  Toyota
Volkswagen          Volkswagen
Volvo                    Volvo
Name: make, dtype: object

### 15.7 Generalization
- yeah I don't know

this one limits the number of categorical values

In [22]:
def generalize_topn(ser, n=5, other='Other'):
    topn = ser.value_counts().index[:n]
    if isinstance(ser.dtype, pd.CategoricalDtype):
        ser = ser.cat.set_categories(
            topn.set_categories(list(topn)+[other]))
        return ser.where(ser.isin(topn), other)

In [25]:
cat_make.pipe(generalize_topn, n=20, other='NA')

0            NA
1            NA
2         Dodge
3         Dodge
4        Subaru
          ...  
41139    Subaru
41140    Subaru
41141    Subaru
41142    Subaru
41143    Subaru
Name: make, Length: 41144, dtype: category
Categories (21, object): ['Chevrolet', 'Ford', 'Dodge', 'GMC', ..., 'Volvo', 'Hyundai', 'Chrysler', 'NA']

and this one is a hierarchical generalization. Suppose want country from make, but only US and German categories and label everything else as Other:

In [26]:
def generalize_mapping(ser, mapping, default):
    seen = None
    res = ser.astype(str)
    for old, new in mapping.items():
        mask = ser.str.contains(old)
        if seen is None:
            seen = mask
        else:
            seen |= mask
        res = res.where(~mask, new)
    res = res.where(seen, default)
    return res.astype('category')


In [27]:
generalize_mapping(cat_make, {'Ford': 'US', 'Telsa': 'US', 'Chevrolet': 'US', 'Dodge': 'US', 'Oldsmobile': 'US', 'Plymouth': 'US', 'BMW': 'German'}, 'Other')

0        Other
1        Other
2           US
3           US
4        Other
         ...  
41139    Other
41140    Other
41141    Other
41142    Other
41143    Other
Name: make, Length: 41144, dtype: category
Categories (3, object): ['German', 'Other', 'US']